In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

In [5]:
#problem 1
hidden_size=1024
batch_size =128

beta1=0.001
beta2=0.001

graph=tf.Graph()

with graph.as_default():
    
    tf_train_dataset=tf.placeholder(np.float32,shape=(batch_size,image_size*image_size))
    tf_train_labels=tf.placeholder(np.float32,shape=(batch_size,num_labels))
    tf_valid_dataset=tf.constant(valid_dataset)
    tf_test_dataset=tf.constant(test_dataset)
    
    weights_1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden_size]))
    biases_1 = tf.Variable(tf.zeros([hidden_size]))
    
    hidden_1=tf.nn.relu(tf.matmul(tf_train_dataset,weights_1)+biases_1)
    
    weights_2 = tf.Variable(
    tf.truncated_normal([hidden_size, num_labels]))
    biases_2 = tf.Variable(tf.zeros([num_labels]))
    
    logits=tf.matmul(hidden_1,weights_2)+biases_2
    loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))+beta1*tf.nn.l2_loss(weights_1)+beta2*tf.nn.l2_loss(weights_2)
    
    optimizer = tf.train.GradientDescentOptimizer(0.1).minimize(loss)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu((tf.matmul(tf_valid_dataset, weights_1) + biases_1)),weights_2)+biases_2)
    test_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu((tf.matmul(tf_test_dataset, weights_1) + biases_1)),weights_2)+biases_2)

In [6]:
num_steps=3001
 
with tf.Session(graph=graph) as session:
    
    tf.global_variables_initializer().run()
    print('initialized')
    
    for step in range(num_steps):
        
        offset=(step*batch_size)%(train_labels.shape[0]-batch_size)
        
        batch_data=train_dataset[offset:(offset+batch_size),:]
        batch_labels=train_labels[offset:(offset+batch_size),:]
        
        feed_dict={tf_train_dataset:batch_data,tf_train_labels:batch_labels}
        
        _,l,predictions=session.run([optimizer,loss,train_prediction],feed_dict=feed_dict)
        
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(
            valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

initialized
Minibatch loss at step 0: 674.389099
Minibatch accuracy: 8.6%
Validation accuracy: 21.6%
Minibatch loss at step 500: 297.076752
Minibatch accuracy: 85.9%
Validation accuracy: 80.5%
Minibatch loss at step 1000: 270.380890
Minibatch accuracy: 80.5%
Validation accuracy: 81.0%
Minibatch loss at step 1500: 235.057877
Minibatch accuracy: 89.8%
Validation accuracy: 81.9%
Minibatch loss at step 2000: 210.104233
Minibatch accuracy: 87.5%
Validation accuracy: 83.1%
Minibatch loss at step 2500: 192.602051
Minibatch accuracy: 85.9%
Validation accuracy: 82.5%
Minibatch loss at step 3000: 171.596283
Minibatch accuracy: 82.0%
Validation accuracy: 83.5%
Test accuracy: 90.5%


In [7]:
#problem 2
    
train_subset=train_dataset[:3000,:]
train_labels_subset=train_labels[:3000,:]

hidden_size=1024
batch_size =10

beta1=0.001
beta2=0.001

graph=tf.Graph()

with graph.as_default():
    
    tf_train_dataset=tf.placeholder(np.float32,shape=(batch_size,image_size*image_size))
    tf_train_labels=tf.placeholder(np.float32,shape=(batch_size,num_labels))
    tf_valid_dataset=tf.constant(valid_dataset)
    tf_test_dataset=tf.constant(test_dataset)
    
    weights_1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden_size]))
    biases_1 = tf.Variable(tf.zeros([hidden_size]))
    
    hidden_1=tf.nn.relu(tf.matmul(tf_train_dataset,weights_1)+biases_1)
    
    weights_2 = tf.Variable(
    tf.truncated_normal([hidden_size, num_labels]))
    biases_2 = tf.Variable(tf.zeros([num_labels]))
    
    logits=tf.matmul(hidden_1,weights_2)+biases_2
    loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))+beta1*tf.nn.l2_loss(weights_1)+beta2*tf.nn.l2_loss(weights_2)
    
    optimizer = tf.train.GradientDescentOptimizer(0.1).minimize(loss)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu((tf.matmul(tf_valid_dataset, weights_1) + biases_1)),weights_2)+biases_2)
    test_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu((tf.matmul(tf_test_dataset, weights_1) + biases_1)),weights_2)+biases_2)

In [8]:
num_steps=3001
 
with tf.Session(graph=graph) as session:
    
    tf.global_variables_initializer().run()
    print('initialized')
    
    for step in range(num_steps):
        
        offset=(step*batch_size)%(train_labels_subset.shape[0]-batch_size)
        
        batch_data=train_subset[offset:(offset+batch_size),:]
        batch_labels=train_labels_subset[offset:(offset+batch_size),:]
        
        feed_dict={tf_train_dataset:batch_data,tf_train_labels:batch_labels}
        
        _,l,predictions=session.run([optimizer,loss,train_prediction],feed_dict=feed_dict)
        
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(
            valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

initialized
Minibatch loss at step 0: 833.507874
Minibatch accuracy: 0.0%
Validation accuracy: 19.0%
Minibatch loss at step 500: 320.093018
Minibatch accuracy: 90.0%
Validation accuracy: 75.9%
Minibatch loss at step 1000: 261.023315
Minibatch accuracy: 90.0%
Validation accuracy: 75.8%
Minibatch loss at step 1500: 235.152954
Minibatch accuracy: 100.0%
Validation accuracy: 80.2%
Minibatch loss at step 2000: 213.150131
Minibatch accuracy: 100.0%
Validation accuracy: 79.6%
Minibatch loss at step 2500: 193.064636
Minibatch accuracy: 100.0%
Validation accuracy: 79.5%
Minibatch loss at step 3000: 194.851303
Minibatch accuracy: 90.0%
Validation accuracy: 80.8%
Test accuracy: 87.8%


In [9]:
#Problem 3
#Dropouts

graph=tf.Graph()

with graph.as_default():
    
    tf_train_dataset=tf.placeholder(np.float32,shape=(batch_size,image_size*image_size))
    tf_train_labels=tf.placeholder(np.float32,shape=(batch_size,num_labels))
    tf_valid_dataset=tf.constant(valid_dataset)
    tf_test_dataset=tf.constant(test_dataset)
    
    weights_1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden_size]))
    biases_1 = tf.Variable(tf.zeros([hidden_size]))
    
    hidden_1=tf.nn.relu(tf.matmul(tf_train_dataset,weights_1)+biases_1)
    
    hidden_1=tf.nn.dropout(hidden_1,0.5)
    
    weights_2 = tf.Variable(
    tf.truncated_normal([hidden_size, num_labels]))
    biases_2 = tf.Variable(tf.zeros([num_labels]))
    
    logits=tf.matmul(hidden_1,weights_2)+biases_2
    loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))+beta1*tf.nn.l2_loss(weights_1)+beta2*tf.nn.l2_loss(weights_2)
    
    optimizer = tf.train.GradientDescentOptimizer(0.1).minimize(loss)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu((tf.matmul(tf_valid_dataset, weights_1) + biases_1)),weights_2)+biases_2)
    test_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu((tf.matmul(tf_test_dataset, weights_1) + biases_1)),weights_2)+biases_2)

In [13]:
num_steps=3001
 
with tf.Session(graph=graph) as session:
    
    tf.global_variables_initializer().run()
    print('initialized')
    
    for step in range(num_steps):
        
        offset=(step*batch_size)%(train_labels_subset.shape[0]-batch_size)
        
        batch_data=train_subset[offset:(offset+batch_size),:]
        batch_labels=train_labels_subset[offset:(offset+batch_size),:]
        
        feed_dict={tf_train_dataset:batch_data,tf_train_labels:batch_labels}
        
        _,l,predictions=session.run([optimizer,loss,train_prediction],feed_dict=feed_dict)
        
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(
            valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

initialized
Minibatch loss at step 0: 957.086853
Minibatch accuracy: 0.0%
Validation accuracy: 19.2%
Minibatch loss at step 500: 690.634094
Minibatch accuracy: 70.0%
Validation accuracy: 72.6%
Minibatch loss at step 1000: 589.081482
Minibatch accuracy: 70.0%
Validation accuracy: 75.3%
Minibatch loss at step 1500: 561.309998
Minibatch accuracy: 70.0%
Validation accuracy: 78.1%
Minibatch loss at step 2000: 366.173706
Minibatch accuracy: 90.0%
Validation accuracy: 75.9%
Minibatch loss at step 2500: 346.056763
Minibatch accuracy: 80.0%
Validation accuracy: 77.0%
Minibatch loss at step 3000: 327.942627
Minibatch accuracy: 80.0%
Validation accuracy: 77.5%
Test accuracy: 83.5%


In [14]:
#Problem 4
hidden_size=1024
hidden2_size=300

batch_size=128

beta1=0.01
beta2=0.01
beta3=0.0001

graph=tf.Graph()

with graph.as_default():
    
    tf_train_dataset=tf.placeholder(np.float32,shape=(batch_size,image_size*image_size))
    tf_train_labels=tf.placeholder(np.float32,shape=(batch_size,num_labels))
    tf_valid_dataset=tf.constant(valid_dataset)
    tf_test_dataset=tf.constant(test_dataset)
    
    weights_1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden_size]))
    biases_1 = tf.Variable(tf.zeros([hidden_size]))
    
    hidden_1=tf.nn.relu(tf.matmul(tf_train_dataset,weights_1)+biases_1)
    
#     hidden_1=tf.nn.dropout(hidden_1,0.5)
    
    weights_2 = tf.Variable(
    tf.truncated_normal([hidden_size, hidden2_size]))
    biases_2 = tf.Variable(tf.zeros([hidden2_size]))
    
    hidden_2=tf.nn.relu(tf.matmul(hidden_1,weights_2)+biases_2)
    
#     hidden_2=tf.nn.dropout(hidden_2,0.5)
  
    weights_3 = tf.Variable(
    tf.truncated_normal([hidden2_size, num_labels]))
    biases_3 = tf.Variable(tf.zeros([num_labels]))
    
    logits=tf.matmul(hidden_2,weights_3)+biases_3
    loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))+beta1*tf.nn.l2_loss(weights_1)+beta2*tf.nn.l2_loss(weights_2)
    +beta3*tf.nn.l2_loss(weights_3)

    
    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.04, global_step,1000,0.90,staircase=True)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
    
#     optimizer = tf.train.GradientDescentOptimizer(0.01).minimize(loss)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu(tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights_1) + biases_1),weights_2)+biases_2),weights_3)
    +biases_3)
    test_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu(tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights_1) + biases_1),weights_2)+biases_2),weights_3)
    +biases_3)

In [15]:
num_steps=9001

with tf.Session(graph=graph) as session:
    
    tf.global_variables_initializer().run()
    print('initialized')
    
    for step in range(num_steps):
        
        offset=(step*batch_size)%(train_dataset.shape[0]-batch_size)
        
        batch_data=train_dataset[offset:(offset+batch_size),:]
        batch_labels=train_labels[offset:(offset+batch_size),:]
        
        feed_dict={tf_train_dataset:batch_data,tf_train_labels:batch_labels}
        
        _,l,predictions=session.run([optimizer,loss,train_prediction],feed_dict=feed_dict)
        
        if (step % 1000 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(
            valid_prediction.eval(), valid_labels))
            
        global_step+=1
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

initialized
Minibatch loss at step 0: 7322.160156
Minibatch accuracy: 10.2%
Validation accuracy: 23.2%
Minibatch loss at step 1000: 1929.709229
Minibatch accuracy: 54.7%
Validation accuracy: 46.9%
Minibatch loss at step 2000: 938.787720
Minibatch accuracy: 71.9%
Validation accuracy: 65.9%
Minibatch loss at step 3000: 491.467773
Minibatch accuracy: 72.7%
Validation accuracy: 72.9%
Minibatch loss at step 4000: 274.428223
Minibatch accuracy: 78.1%
Validation accuracy: 76.7%
Minibatch loss at step 5000: 162.467880
Minibatch accuracy: 82.0%
Validation accuracy: 79.9%
Minibatch loss at step 6000: 101.577263
Minibatch accuracy: 79.7%
Validation accuracy: 81.6%
Minibatch loss at step 7000: 66.535233
Minibatch accuracy: 82.8%
Validation accuracy: 82.6%
Minibatch loss at step 8000: 45.775497
Minibatch accuracy: 79.7%
Validation accuracy: 83.6%
Minibatch loss at step 9000: 32.525505
Minibatch accuracy: 83.6%
Validation accuracy: 84.0%
Test accuracy: 90.8%
